# Capstone Two #

## Predicting Test Performance for Specialised High School Admissions ##

Performance on the Specialized High School Admissions Test (SHSAT) determines eligibility to one of the eight specialized high schools in New York City. It is administered by the New York City Department of Education (DOE) to about a third of the city’s 8th graders, with 5,000 receiving admissions offers. Of major concern is the racial & ethnic breakdown of admitted students, showing significant underrepresentation from black and latinx students. 

The goal of this analysis is to elicit which factors predict performance on the SHSAT. These factors will serve as beacons to direct or draw services, whether education-based or otherwise, towards improving the percentage of black and latinx students admitted to the specialized high schools. 

This approach aims to quantify which variables lead to admissions offers beyond prior proxies: English Language Learners, Students with Disabilities, Students on Free/Reduced Lunch, and Students with Temporary Housing.

In [23]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import urllib.request
%matplotlib inline

### Data Collection & Loading Data Sets ###

Two main data sets will be used:

*2016 School Explorer*  
This dataset consists of 1272 schools in New York city, and 161 variables, provided via kaggle. Primarily, it’s school descriptors, e.g. grades, race & ethnicity student percentages, high/low performing percentages of students. Data is available as a single csv file.

*2017-2018 SHSAT Admissions Test Offers By Sending School*  
This dataset consists of the 2017 SHSAT results, published by NYC in 2018. All test takers are north of 28,000, grade 8 students, Test takers and offers received are grouped by school. Data is available as a single csv file from the NYC Open Data portal.

In [8]:
#downloading the 2016 School Explorer data set to raw folder
url = 'https://storage.googleapis.com/kagglesdsdata/datasets%2F33225%2F44131%2F2016%20School%20Explorer.csv?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1595273466&Signature=fk6%2BI64ZSKXeenOP24Hsst9gNp2z3gFDGnz1rSXzcrnS874EFfR1VjUPVu0mCoN0bwXxJ7udjKpGlD51QLqiolRTpt9t%2F6ko672nNzd2KU0zJd4xRN8yW4Ouk1XxbCCTN2u6In241T1%2BY1RMpSp5rQgko83zQtwPClQPsl%2BWynlztsHV1aWF2K1J6MUy1SBaXyHvTSXBiMp1G2FvCoVjRVyjkXwV94Xgayi8Zgs3ISjyVUZn3yYzuyarl8NUwSnryWnfCE1debgt5z9AP5aTv7IbUA297hpYAhHZR0NjtKMwoadxypbWbBZ6cUTgI8KT4L4q8LdCgJ6SDJolkJERaQ%3D%3D'
file_explorer = '../data/raw/2016_School_Explorer.csv'
urllib.request.urlretrieve(url, file_explorer)

#downloading the 2017-2018 SHSAT data set to raw folder
url = 'https://data.cityofnewyork.us/resource/vsgi-eeb5.csv'
file_offers = '../data/raw/2017-2018_SHSAT_Admissions_Test_Offers_By_Sending_School.csv'
urllib.request.urlretrieve(url, file_offers)

In [9]:
#loading data sets
df_explorer = pd.read_csv(file_explorer)
df_offers = pd.read_csv(file_offers)

In [19]:
df_explorer.head()

,Adjusted Grade,New?,Other Location Code in LCGMS,School Name,SED Code,Location Code,District,Latitude,Longitude,Address (Full),...,Grade 8 Math - All Students Tested,Grade 8 Math 4s - All Students,Grade 8 Math 4s - American Indian or Alaska Native,Grade 8 Math 4s - Black or African American,Grade 8 Math 4s - Hispanic or Latino,Grade 8 Math 4s - Asian or Pacific Islander,Grade 8 Math 4s - White,Grade 8 Math 4s - Multiracial,Grade 8 Math 4s - Limited English Proficient,Grade 8 Math 4s - Economically Disadvantaged
0,NaN,NaN,NaN,P.S. 015 ROBERTO CLEMENTE,310100010015,01M015,1,40.721834,-73.978766,"333 E 4TH ST NEW YORK, NY 10009",...,0,0,0,0,0,0,0,0,0,0
1,NaN,NaN,NaN,P.S. 019 ASHER LEVY,310100010019,01M019,1,40.729892,-73.984231,"185 1ST AVE NEW YORK, NY 10003",...,0,0,0,0,0,0,0,0,0,0
2,NaN,NaN,NaN,P.S. 020 ANNA SILVER,310100010020,01M020,1,40.721274,-73.986315,"166 ESSEX ST NEW YORK, NY 10002",...,0,0,0,0,0,0,0,0,0,0
3,NaN,NaN,NaN,P.S. 034 FRANKLIN D. ROOSEVELT,310100010034,01M034,1,40.726147,-73.975043,"730 E 12TH ST NEW YORK, NY 10009",...,48,1,0,0,0,0,0,0,0,0
4,NaN,NaN,NaN,THE STAR ACADEMY - P.S.63,310100010063,01M063,1,40.724404,-73.986360,"121 E 3RD ST NEW YORK, NY 10009",...,0,0,0,0,0,0,0,0,0,0


In [20]:
df_offers.head()

,feeder_school_dbn,feeder_school_name,count_of_students_in_hs_admissions,count_of_testers,count_of_offers
0,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,58.0,6,0-5
1,01M140,P.S. 140 NATHAN STRAUS,67.0,6,0-5
2,01M184,P.S. 184M SHUANG WEN,88.0,67,23
3,01M188,P.S. 188 THE ISLAND SCHOOL,59.0,0-5,0-5
4,01M301,"TECHNOLOGY, ARTS, AND SCIENCES STUDIO",51.0,11,0-5


### Data Definitions ###

Generally exploring both datasets to understand what's in them.

In [28]:
df_explorer.columns

Index(['Adjusted Grade', 'New?', 'Other Location Code in LCGMS', 'School Name',
       'SED Code', 'Location Code', 'District', 'Latitude', 'Longitude',
       'Address (Full)',
       ...
       'Grade 8 Math - All Students Tested', 'Grade 8 Math 4s - All Students',
       'Grade 8 Math 4s - American Indian or Alaska Native',
       'Grade 8 Math 4s - Black or African American',
       'Grade 8 Math 4s - Hispanic or Latino',
       'Grade 8 Math 4s - Asian or Pacific Islander',
       'Grade 8 Math 4s - White', 'Grade 8 Math 4s - Multiracial',
       'Grade 8 Math 4s - Limited English Proficient',
       'Grade 8 Math 4s - Economically Disadvantaged'],
      dtype='object', length=161)

In [30]:
df_explorer.shape

(1272, 161)

In [34]:
df_explorer.nunique

<bound method DataFrame.nunique of      Adjusted Grade New? Other Location Code in LCGMS  \
0               NaN  NaN                          NaN   
1               NaN  NaN                          NaN   
2               NaN  NaN                          NaN   
3               NaN  NaN                          NaN   
4               NaN  NaN                          NaN   
...             ...  ...                          ...   
1267            NaN  NaN                          NaN   
1268            NaN  NaN                          NaN   
1269            NaN  NaN                          NaN   
1270            NaN  NaN                          NaN   
1271            NaN  NaN                          NaN   

                                   School Name      SED Code Location Code  \
0                    P.S. 015 ROBERTO CLEMENTE  310100010015        01M015   
1                          P.S. 019 ASHER LEVY  310100010019        01M019   
2                         P.S. 020 ANNA SILVER

We can tell that there are 161 variables in the 2016 School Explorer data set with 1,272 rows of data consisting of physical descriptors and descriptive statistics information for all the schools.

In [27]:
df_offers.columns

Index(['feeder_school_dbn', 'feeder_school_name',
       'count_of_students_in_hs_admissions', 'count_of_testers',
       'count_of_offers'],
      dtype='object')

In [31]:
df_offers.shape

(594, 5)

In [35]:
df_offers.nunique

<bound method DataFrame.nunique of     feeder_school_dbn                     feeder_school_name  \
0              01M034         P.S. 034 FRANKLIN D. ROOSEVELT   
1              01M140                 P.S. 140 NATHAN STRAUS   
2              01M184                   P.S. 184M SHUANG WEN   
3              01M188             P.S. 188 THE ISLAND SCHOOL   
4              01M301  TECHNOLOGY, ARTS, AND SCIENCES STUDIO   
..                ...                                    ...   
589            84X538                 ICAHN CHARTER SCHOOL 5   
590            84X703       BRONX PREPARATORY CHARTER SCHOOL   
591            84X704            KIPP ACADEMY CHARTER SCHOOL   
592            84X706          HARRIET TUBMAN CHARTER SCHOOL   
593            84X717                   ICAHN CHARTER SCHOOL   

     count_of_students_in_hs_admissions count_of_testers count_of_offers  
0                                  58.0                6             0-5  
1                                  67.0       

The 2017-2018 SHSAT Admissions Test Offers By Sending School has 5 variables with 594 rows of data consisting of names and descriptive statistics for all of the schools. This makes sense because not all schools send students to the SHSAT.